In [9]:
import sendgrid
import os
from sendgrid.helpers.mail import *
import pyrebase
import pymongo
import datetime
import pprint
from twilio.rest import Client
from bson import ObjectId

Below is the code to to the local mongo db database. Mongo DB has to be installed on the machine for this to work, and there should be a db initialized as "test database". There are helper methods below to help make a user, or to print all current users. A user object looks like this:

{'_id': ObjectId('5cc791eddd17d02e7d8ae0d4'),
 'control_vectors': [],
 'email': 'pmakkar97@gmail.com',
 'first_name': 'Pal',
 'last_name': 'Makkar',
 'phone': '4086246734',
 'states': [],
 'time_added': datetime.datetime(2019, 4, 30, 0, 8, 13, 840000)
}

In [2]:
def get_db():
	client = pymongo.MongoClient('mongodb://localhost:27017/')
	db = client.test_database

	return db


In [3]:
db = get_db()

In [44]:
def add_user(first_name, last_name, email, phone):
	user = {"first_name": first_name, "last_name": last_name, "email": email,"phone": phone, "time_added": datetime.datetime.utcnow(), "control_vectors": [], "states": []}
	user_id = db.users.insert_one(user).inserted_id
	return user_id

In [45]:
add_user("Pal", "Makkar", "pmakkar97@gmail.com", "4086246734")
pprint.pprint(db.users.find_one())


{'_id': ObjectId('5cc791eddd17d02e7d8ae0d4'),
 'control_vectors': [],
 'email': 'pmakkar97@gmail.com',
 'first_name': 'Pal',
 'last_name': 'Makkar',
 'phone': '4086246734',
 'states': [],
 'time_added': datetime.datetime(2019, 4, 30, 0, 8, 13, 840000)}


In [47]:
def print_users():
	for x in db.users.find():
  		print(x)

In [48]:
print_users()

{'_id': ObjectId('5cc791eddd17d02e7d8ae0d4'), 'first_name': 'Pal', 'last_name': 'Makkar', 'email': 'pmakkar97@gmail.com', 'phone': '4086246734', 'time_added': datetime.datetime(2019, 4, 30, 0, 8, 13, 840000), 'control_vectors': [], 'states': []}


Below are helper methods to send an email or text message. The email helper uses the Sendgrid API, and the messaging helper uses the Twilio. An account must be set up for whichever one is used, and the appropriate authentication keys variables must be assigned correctly before use. For Twilio, this is the account_sid and auth_token variables. One secure way to use auth tokens for this purpose is to have a file named "keys.env" with the lines 
"export SENDGRID_API_KEY='fill in sendgrid key'" and
"export TWILIO_AUTH='fill in twilio key'",
and then run "source keys.env" in the terminal to store these as environment variables for use in the code below.

In [8]:
def send_email(from_address, to_address, subject, text):
	sg = sendgrid.SendGridAPIClient(apikey=os.environ.get('SENDGRID_API_KEY'))
	from_email = Email(from_address)
	to_email = Email(to_address)
	content = Content("text/plain", text)
	mail = Mail(from_email, subject, to_email, content)
	response = sg.client.mail.send.post(request_body=mail.get())
	print(response.status_code)
	print(response.body)
	print(response.headers)

In [9]:
def send_test_email():
	from_address = "test@testing.com"
	to_address = "pmakkar97@gmail.com"
	subject = "test"
	text = "test"

	send_email(from_address, to_address, subject, text)

In [30]:
send_test_email()

202
b''
Server: nginx
Date: Mon, 29 Apr 2019 23:18:34 GMT
Content-Length: 0
Connection: close
X-Message-Id: _S0rHBVrQnqEeTKLv3CsIQ
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html




In [7]:
def send_text(from_number, to_number, body):
	account_sid = 'ACff26b77b06fd2bd97795b331ef0d8d25'
	auth_token = os.environ.get('TWILIO_AUTH')
	client = Client(account_sid, auth_token)

	message = client.messages.create(
		body=body,
		from_=from_number,
		to=to_number
	)
	return message.sid



In [12]:
def send_test_text():
	print(send_text('+15623860475', '+14086246734', "Join Earth's mightiest heroes."))
    
send_test_text()

SM46ff05f146974d5fafdec4206fcf08df


The below cells help add or remove a control vector or state for a user, and the schema for these is flexible. 

In [76]:
def add_control_vector(user_id, control_vector):

    db.users.update_one(
       { "_id": user_id },
       { "$push": { "control_vectors": control_vector } }
    )
    
def pop_control_vector(user_id):
    db.users.update_one( { "_id": user_id }, { "$pop": { "control_vectors": 1 } } )
    
    
user_id = ObjectId('5cc791eddd17d02e7d8ae0d4')
cv = {"time_iteration": 1, "cv_values": {"Python": (8, 1,10), "Project": (5,1,5)}, "cv_descriptions": {"Python": "Ability to use Python (1-10)", "Project": "State of project completion (1-5)"}}

      
add_control_vector(ObjectId('5cc791eddd17d02e7d8ae0d4'), cv)


print(db.users.find_one())
      

{'_id': ObjectId('5cc791eddd17d02e7d8ae0d4'), 'first_name': 'Pal', 'last_name': 'Makkar', 'email': 'pmakkar97@gmail.com', 'phone': '4086246734', 'time_added': datetime.datetime(2019, 4, 30, 0, 8, 13, 840000), 'control_vectors': [{'time_iteration': 1, 'cv_values': {'Python': [8, 1, 10], 'Project': [5, 1, 5]}, 'cv_descriptions': {'Python': 'Ability to use Python (1-10)', 'Project': 'State of project completion (1-5)'}}], 'states': [{'time_iteration': 1, 'state_values': {'Python': [2, 1, 10], 'Project': [1, 1, 5]}, 'time': datetime.datetime(2019, 4, 30, 1, 14, 2, 81000), 'notes': ''}]}


In [75]:
def add_state(user_id, state):
    db.users.update_one(
       { "_id": user_id },
       { "$push": { "states": state } }
    )
    
def pop_state(user_id):
    db.users.update_one( { "_id": user_id }, { "$pop": { "state": 1 } } )
    
state = {
    "time_iteration": 1,
    "state_values": {"Python": (2, 1,10), "Project": (1,1,5)},
    "time":  datetime.datetime.utcnow(),
    "notes": ""
}

add_state(ObjectId('5cc791eddd17d02e7d8ae0d4'), state)
print_users()



{'_id': ObjectId('5cc791eddd17d02e7d8ae0d4'), 'first_name': 'Pal', 'last_name': 'Makkar', 'email': 'pmakkar97@gmail.com', 'phone': '4086246734', 'time_added': datetime.datetime(2019, 4, 30, 0, 8, 13, 840000), 'control_vectors': [], 'states': [{'time_iteration': 1, 'state_values': {'Python': [2, 1, 10], 'Project': [1, 1, 5]}, 'time': datetime.datetime(2019, 4, 30, 1, 14, 2, 81000), 'notes': ''}]}


The below code sets up a flask server to run the sample demo. The demo requires you to run this code to start up the server, then you can go to "http://localhost:5000/" in a browser, where there is a form to be filled put twith the new user information. Once the form is submitted, a new user is created and they will recieve a text with a target number. Note that from here, the flask server handles incoming messages via Twilio, and needs to have a publically accessible URL. So it needs to be run with ngrok or hosted somewhere on the cloud, with the publically acessible URL set up in the Twilio's recieveing webhooks settings. From there, the user can text back the current number, and the program will respond with whether they should increase the current number, decrease it, or if they've reached the goal. For now, each time the user texts a new number, it is stored as state vector as a tuple with the target, and a control vector is stored with which direction to go along with the target as a tuple.

In [ ]:
@app.route('/')
def student():
   return render_template('student.html')


@app.route('/result',methods = ['POST', 'GET'])
def result():
   if request.method == 'POST':
      result = request.form
      db = get_db()
      target = random.randint(1,101)
      add_user(db, result['first'], result['last'], result['email'], result['phone'], target)
      
      send_text(result['phone'], "Welcome " + result['first'] + "! Your target number is " + str(target) + ". Send me your current number")
      return render_template("result.html",result = result)


@app.route("/sms", methods=['GET', 'POST'])
def sms_ahoy_reply():
    """Respond to incoming messages and put user into the system."""

    from_number = request.form['From']
    to_number = request.form['To']
    body = request.form['Body']
    curr = int(body)
    db = get_db()
    user_id = db.users.find_one({"phone":from_number})['_id']
    target = db.users.find_one({"phone":from_number})['target']
    add_state(db, user_id, [curr, target])

    resp = MessagingResponse()


    if curr < target:
        step = "Increase"
    elif curr == target:
        step = "You're done!"
    else:
        step = "Decrease"

    add_control_vector(db, user_id, [step, target])


    resp.message(step)

    return str(resp)


if __name__ == '__main__':
   app.run(debug = True)